<a href="https://colab.research.google.com/github/KPantelidis/trendlines/blob/main/Clean_Code_(kinda).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##READ ME

####Files
Three csv files are necessary to run the code
- `Diff_to_DSSP_LSQ_LC.csv`: Includes all the DSSP (as well as LSQ & LC information for each protein from the 83 protein set
- `proteins83_PCDDB.csv`: The real/experimental CD spectra for all 83 proteins from Wavelength 180nm to 260nm  
- `proteins83_PDBMD2CD.csv`: The PDBMD2CD prediction for all 83 proteins from Wavelength 180nm to 260nm 

At this stage, we are testing the potential "good" trendlines to find a good basis set of trendlines, so in the future there will be another .csv file that includes all the requested trendlines.\
So, at the moment, the work for the trendlines is done in another script and presented here as `trendline_list`  \
*e.g. the current trendline list has the best 5 trendlines that have 1ADO protein as protein2 in the trendline*

####Functions
The code has two big functions and a smaller one:
- `getLinearReg2D`: which gets a list of 4 proteins and generates the line of 
best fit/linear regression or as we say the *Trendline*
- `point_on_line`: A small generic function that gives back the projected point of any point into a line
- `getFinalPredictionTable`: which gets 
  (1) a table of the 4 proteins with their DSSP TOTAL ALPHA & BETA values as x and y and
  (2) the linear regression that the previous function generates.

  The function projects the proteins to the trendline (using `point_on_line` function from above) and then calculates the 
distances between the projected points.\
  Now, at this stage, we are looking at the relationship of **protein2** to the other three proteins and so the function calculates the percentage distance of protein2 to the other proteins.\
  Then, using the PCDDB table, it calculates predict1 and predict2\
  **predict1**: prediction of protein2 in relationship **protein1-protein2-protein3**\
  **predict2**: prediction of protein2 in relationship **protein1-protein2-protein4**\
  Finally, **prediction** column is calculated as the average of predict1 & predict2

####Plots
After every Final Prediction Table there is a plot of 5 lines 
Again, at this stage we are checking the relationship of protein 2 to the others and this is the only real CD spectra (blue line), as all the others are predictions.\
All plots are interactive, using `plotly` package

**Experimental: Blue Line\
PDBMD2CD prediction: Green Line\
predict1: Orange Line\
predict2: Yellow Line\
prediction: Red Line**


In [ ]:
#@title Import packages

import pandas as pd
import numpy as np
from numpy.linalg import norm
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
#@title Read DSSP, PCDDB, PDBMD2CD data
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Diff_to_DSSP_LSQ_LC.csv")
#print(df)

## keep helix and beta only - 2 dimensions
df = pd.concat([df['protein'],df['DSSP_H']+df['DSSP_G']+df['DSSP_I'], 
                df['DSSP_E']], 
               axis=1,keys = ('protein', 'TOTAL_ALPHA', 'TOTAL_BETA'))
#print(df)


## read experimental data, Wavelength for each protein
df_PCDDB = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/proteins83_PCDDB.csv")
#print(df_PCDDB)



df_PDBMD2CD = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/proteins83_PDBMD2CD.csv")
#print(df_PDBMD2CD)

In [ ]:
#@title Requested Trendlines (CURRENTLY EXPLORNG 1ADO in position 2)

#The trendlines have been calculated in another script to avoid volume of information
#  There are 5 trendlines, all associated with protein 1ADO
# 1ADO had 88560 trendlines
# From these, 21518 had 1ADO in position protein2 (ORDERED BY HELIX)
# Then the avg_distance was used to find the 5 best

trendline_list=[['3RN3','1ADO','2DYR','2NOP'],\
        ['1NLS','1ADO','2DYR','2NOP'],\
        ['3RN3','1ADO','1LIN','2NOP'],\
        ['1NLS','1ADO','1LIN','2NOP'],\
        ['1OFS','1ADO','2DYR','2NOP']]
#trendline_list

In [ ]:
#@title getLinearReg2D

def getLinearReg2D(trendline_list):
    ''' creates a function "getLinearReg2D" that takes a list of 4 proteins as input and 
      gives back the line of best fit/linear regression as a table of x and y for φ points
    ''' 
    ## Select proteins
    protein1 = trendline_list[0]
    protein2 = trendline_list[1]
    protein3 = trendline_list[2]
    protein4 = trendline_list[3]
    #print(protein1, protein2, protein3,protein4)


    # create H,E,C, Diff dataframe
    df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Diff_to_DSSP_LSQ_LC.csv")
    #print(df)
    df1 = pd.concat([df['protein'],df['DSSP_H']+df['DSSP_G']+df['DSSP_I'], 
                df['DSSP_E']], 
               axis=1,keys = ('protein', 'TOTAL_ALPHA', 'TOTAL_BETA'))


    # Linear regression    

    #protein1
    xx1 = float(df1[df1['protein']==protein1]['TOTAL_ALPHA'])
    yy1 = float(df1[df1['protein']==protein1]['TOTAL_BETA'])
    xy1 = [protein1, xx1, yy1]

    #protein2
    xx2 = float(df1[df1['protein']==protein2]['TOTAL_ALPHA'])
    yy2 = float(df1[df1['protein']==protein2]['TOTAL_BETA'])
    xy2 = [protein2, xx2, yy2]

    #protein3
    xx3 = float(df1[df1['protein']==protein3]['TOTAL_ALPHA'])
    yy3 = float(df1[df1['protein']==protein3]['TOTAL_BETA'])
    xy3 = [protein3, xx3, yy3]

    #protein4
    xx4 = float(df1[df1['protein']==protein4]['TOTAL_ALPHA'])
    yy4 = float(df1[df1['protein']==protein4]['TOTAL_BETA'])
    xy4 = [protein4, xx4, yy4]


    ## work for linear regression starts here
    df2 = pd.DataFrame([xy1, xy2 , xy3, xy4],
                    columns=['protein','TOTAL_ALPHA', 'TOTAL_BETA'])

    X = df2[['TOTAL_ALPHA', 'TOTAL_BETA']].values
    Xlen = X.shape[0]
    avgPointCloud = 1 / Xlen * np.array([np.sum(X[:, 0]), np.sum(X[:, 1])])
    Xmean = X - avgPointCloud
    cov = 1 / Xlen * X.T.dot(Xmean)
    t = np.arange(-5, 5, 1)
    linearReg = avgPointCloud + cov[:, 0] * np.vstack(t)
    
    ## return the φ points of the linear regression 
    df_linearReg = pd.DataFrame()
    df_linearReg['x'] = linearReg[:, 0]
    df_linearReg['y'] = linearReg[:, 1]
    return df_linearReg
    

In [ ]:
#@title getFinalPredictionTable 
## function to project a point p to a line a-b
## which is used in the getFinalTable function
def point_on_line(a, b, p):
    ap = p - a
    ab = b - a
    result = a + np.dot(ap, ab) / np.dot(ab, ab) * ab
    return result


def getFinalPredictionTable(df_linearReg, df_trendline_proteins):
    
    '''
    creates a function that gets the linear regression and the proteins of a trendline
    and gives back the final prediction table
    '''
    
    A = np.array([df_linearReg['x'].iloc[0], df_linearReg['y'].iloc[0]])
    B = np.array([df_linearReg['x'].iloc[-1], df_linearReg['y'].iloc[-1]])


    P1 = np.array([float(df_trendline_proteins['TOTAL_ALPHA'].iloc[0]), float(df_trendline_proteins['TOTAL_BETA'].iloc[0])])
    P2 = np.array([float(df_trendline_proteins['TOTAL_ALPHA'].iloc[1]), float(df_trendline_proteins['TOTAL_BETA'].iloc[1])])
    P3 = np.array([float(df_trendline_proteins['TOTAL_ALPHA'].iloc[2]), float(df_trendline_proteins['TOTAL_BETA'].iloc[2])])
    P4 = np.array([float(df_trendline_proteins['TOTAL_ALPHA'].iloc[3]), float(df_trendline_proteins['TOTAL_BETA'].iloc[3])])


    projected_protein1 = point_on_line(A, B, P1)
    projected_protein2 = point_on_line(A, B, P2)
    projected_protein3 = point_on_line(A, B, P3)
    projected_protein4 = point_on_line(A, B, P4)

    # Calculate Euclidean distance
    dist_p2_p3 = np.linalg.norm(projected_protein2-projected_protein3)
    #print('Distance between protein2 & protein3: ',dist_p2_p3)

    dist_p2_p1 = np.linalg.norm(projected_protein2-projected_protein1)
    #print('Distance between protein2 & protein1: ',dist_p2_p1)

    dist_p2_p4 = np.linalg.norm(projected_protein2-projected_protein4)
    #print('Distance between protein2 & protein4: ',dist_p2_p4)


    #========
    dist_p1_p3 = np.linalg.norm(projected_protein1-projected_protein3)
    #print('Distance between protein1 & protein3: ',dist_p1_p3)

    dist_p1_p4 = np.linalg.norm(projected_protein1-projected_protein4)
    #print('Distance between protein1 & protein4: ',dist_p1_p4)

    dist_p3_p4 = np.linalg.norm(projected_protein3-projected_protein4)
    #print('Distance between protein3 & protein4: ',dist_p3_p4)
    #========


    ###   P1 P2 P3
    dist_p2_p3_percent = dist_p2_p3/dist_p1_p3*100
    dist_p2_p1_percentA = dist_p2_p1/dist_p1_p3*100

    ###   P1 P2 P4
    dist_p2_p1_percentB = dist_p2_p1/dist_p1_p4*100
    dist_p2_p4_percent = dist_p2_p4/dist_p1_p4*100

    ###   P2 P3 P4 
    dist_p2_p3_percentC = dist_p2_p3/dist_p3_p4*100
    dist_p2_p4_percentC = dist_p2_p4/dist_p3_p4*100


    #print('protein2 from protein3: ', '%.2f'  %dist_p2_p3_percent, '%')
    #print('protein2 from protein1: ', '%.2f'  %dist_p2_p1_percentA, '%')
    #print('\n')
    #print('protein2 from protein1: ', '%.2f'  %dist_p2_p1_percentB, '%')
    #print('protein2 from protein4: ', '%.2f'  %dist_p2_p4_percent, '%')
    #print('\n')
    #print('protein2 from protein3: ', '%.2f'  %dist_p2_p3_percentC, '%')
    #print('protein2 from protein4: ', '%.2f'  %dist_p2_p4_percentC, '%')
    
    
    proteins_wv_table = pd.concat([df_PCDDB.iloc[:,0], \
                             df_PCDDB[df_trendline_proteins['protein'][0]],\
                             df_PCDDB[df_trendline_proteins['protein'][1]],\
                             df_PCDDB[df_trendline_proteins['protein'][2]],\
                             df_PCDDB[df_trendline_proteins['protein'][3]]] ,axis=1)
    
    
    final_table = pd.concat([proteins_wv_table.iloc[:,0], proteins_wv_table.iloc[:,2]], axis=1, \
                          keys = ('Wavelength', 'Experimental'))
    final_table['PDBMD2CD'] = df_PDBMD2CD[df_trendline_proteins['protein'][1]]
    final_table['predict1'] = (dist_p2_p3_percent*proteins_wv_table.iloc[:,1]/100) + (dist_p2_p1_percentA*proteins_wv_table.iloc[:,3]/100)
    final_table['predict2'] = (dist_p2_p4_percent*proteins_wv_table.iloc[:,1]/100) + (dist_p2_p1_percentB*proteins_wv_table.iloc[:,4]/100)
    final_table['Prediction'] = (final_table['predict1'] + final_table['predict2'])/2

    return final_table

In [ ]:
############ WORK STARTS HERE ########################

for m in trendline_list:
    print('Trendline', m)
    dataframe=[]
    for a in m:
        df0 = df[df['protein']==a]
        dataframe.append(df0)
        
    df_trendline_proteins = pd.concat(dataframe, axis=0, ignore_index=True)   
    df_trendline_proteins = df_trendline_proteins.sort_values('TOTAL_ALPHA')
    df_trendline_proteins = df_trendline_proteins.set_index([pd.Index([0, 1, 2, 3])])
    print('\n',df_trendline_proteins)
    
    df_linearReg = getLinearReg2D(m)
    final_table = getFinalPredictionTable(df_linearReg, df_trendline_proteins)
    print('\n',final_table)
    
    
    
    ####### PLOTS##################
    fig = px.line(final_table, x='Wavelength', y=['Experimental', 'predict1', 'predict2', 'Prediction', 'PDBMD2CD'], \
                  color_discrete_sequence=["blue", "orange", "yellow", "red", "green"])
 
    fig.show()
    fig.write_html("/content/drive/MyDrive/Colab Notebooks/1ADO_asProtein2_Best5_.html", auto_open = True)
 

Trendline ['3RN3', '1ADO', '2DYR', '2NOP']

   protein  TOTAL_ALPHA  TOTAL_BETA
0    3RN3        0.209       0.331
1    1ADO        0.458       0.138
2    2DYR        0.570       0.051
3    2NOP        0.625       0.009

     Wavelength  Experimental  PDBMD2CD  predict1  predict2  Prediction
0          180      2.193570  1.852901  0.057598  2.562784    1.310191
1          181      2.690140  2.365682  0.404090  3.170814    1.787452
2          182      3.218490  2.925980  0.864110  3.757267    2.310689
3          183      3.633880  3.455137  1.352118  4.174922    2.763520
4          184      4.277670  4.140603  1.875236  4.677875    3.276555
..         ...           ...       ...       ...       ...         ...
76         256     -0.061940 -0.045093 -0.020477 -0.016165   -0.018321
77         257     -0.030455 -0.026231 -0.055192 -0.026523   -0.040858
78         258     -0.045236 -0.027888 -0.050544 -0.036034   -0.043289
79         259     -0.007657 -0.005383  0.004164  0.003974    0.0040

Trendline ['1NLS', '1ADO', '2DYR', '2NOP']

   protein  TOTAL_ALPHA  TOTAL_BETA
0    1NLS        0.038       0.460
1    1ADO        0.458       0.138
2    2DYR        0.570       0.051
3    2NOP        0.625       0.009

     Wavelength  Experimental  PDBMD2CD  predict1  predict2  Prediction
0          180      2.193570  1.852901 -0.652685  2.090250    0.718782
1          181      2.690140  2.365682 -0.245541  2.830780    1.292619
2          182      3.218490  2.925980  0.308065  3.567969    1.938017
3          183      3.633880  3.455137  0.891828  4.101149    2.496488
4          184      4.277670  4.140603  1.534688  4.761707    3.148198
..         ...           ...       ...       ...       ...         ...
76         256     -0.061940 -0.045093 -0.040520 -0.042365   -0.041442
77         257     -0.030455 -0.026231 -0.079161 -0.053297   -0.066229
78         258     -0.045236 -0.027888 -0.069963 -0.059416   -0.064690
79         259     -0.007657 -0.005383 -0.010119 -0.015329   -0.0127

Trendline ['3RN3', '1ADO', '1LIN', '2NOP']

   protein  TOTAL_ALPHA  TOTAL_BETA
0    3RN3        0.209       0.331
1    1ADO        0.458       0.138
2    1LIN        0.568       0.054
3    2NOP        0.625       0.009

     Wavelength  Experimental  PDBMD2CD  predict1  predict2  Prediction
0          180      2.193570  1.852901  1.633164  2.562783    2.097973
1          181      2.690140  2.365682  1.994310  3.170813    2.582561
2          182      3.218490  2.925980  2.433151  3.757265    3.095208
3          183      3.633880  3.455137  2.900498  4.174920    3.537709
4          184      4.277670  4.140603  3.516832  4.677873    4.097352
..         ...           ...       ...       ...       ...         ...
76         256     -0.061940 -0.045093  0.032381 -0.016165    0.008108
77         257     -0.030455 -0.026231  0.083676 -0.026523    0.028576
78         258     -0.045236 -0.027888  0.080031 -0.036034    0.021999
79         259     -0.007657 -0.005383 -0.002617  0.003974    0.0006

Trendline ['1NLS', '1ADO', '1LIN', '2NOP']

   protein  TOTAL_ALPHA  TOTAL_BETA
0    1NLS        0.038       0.460
1    1ADO        0.458       0.138
2    1LIN        0.568       0.054
3    2NOP        0.625       0.009

     Wavelength  Experimental  PDBMD2CD  predict1  predict2  Prediction
0          180      2.193570  1.852901  1.158497  2.090253    1.624375
1          181      2.690140  2.365682  1.581201  2.830783    2.205992
2          182      3.218490  2.925980  2.108854  3.567973    2.838414
3          183      3.633880  3.455137  2.667215  4.101152    3.384184
4          184      4.277670  4.140603  3.414140  4.761711    4.087925
..         ...           ...       ...       ...       ...         ...
76         256     -0.061940 -0.045093  0.020170 -0.042365   -0.011097
77         257     -0.030455 -0.026231  0.079737 -0.053297    0.013220
78         258     -0.045236 -0.027888  0.079386 -0.059416    0.009985
79         259     -0.007657 -0.005383 -0.017583 -0.015329   -0.0164

Trendline ['1OFS', '1ADO', '2DYR', '2NOP']

   protein  TOTAL_ALPHA  TOTAL_BETA
0    1OFS        0.041       0.464
1    1ADO        0.458       0.138
2    2DYR        0.570       0.051
3    2NOP        0.625       0.009

     Wavelength  Experimental  PDBMD2CD  predict1  predict2  Prediction
0          180      2.193570  1.852901 -0.910940  1.742030    0.415545
1          181      2.690140  2.365682 -0.478190  2.517113    1.019461
2          182      3.218490  2.925980  0.082970  3.264501    1.673736
3          183      3.633880  3.455137  0.698524  3.840560    2.269542
4          184      4.277670  4.140603  1.358414  4.524092    2.941253
..         ...           ...       ...       ...       ...         ...
76         256     -0.061940 -0.045093 -0.038743 -0.039969   -0.039356
77         257     -0.030455 -0.026231 -0.071311 -0.042708   -0.057009
78         258     -0.045236 -0.027888 -0.064620 -0.052208   -0.058414
79         259     -0.007657 -0.005383 -0.010766 -0.016202   -0.0134